# Dev 06 - Preview

In [2]:
import os
import glob
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import toshi
import cspec

zmap = cspec.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

# filesize = os.path.getsize(file)

In [11]:
files = []
for day in glob.glob('/Volumes/Transcend/akita/IQ/IQdata/*'):
    for hour in glob.glob('{}/*'.format(day)):
        #print(day, hour)
        for file in glob.glob('{}/*.dat'.format(hour)):
            files.append(file)

In [13]:
len(files)

4675

In [19]:
for k in range(0, len(files), 20):
    file = files[k]
    print(file)
    
    s = time.time()
    ray_pulses, cpi_headers = toshi.read(file)
    e = time.time()
    print('Data read in {:.2f} s'.format(e - s))

    # Go through the pulses for azimuth
    a = np.zeros(len(ray_pulses), dtype=np.single)
    for k, pulses in enumerate(ray_pulses):
        a[k] = pulses[0].azimuth

    # Choose ray 2 to whatever that completes the 360-deg coverage
    n = np.argmin(np.abs(a[3:] - a[2])) + 1
    a = a[2:n+2]

    # Subsetting
    ray_pulses = ray_pulses[2:n+2]
    cpi_headers = cpi_headers[2:n+2]
    
    # Dimensions
    naz = len(ray_pulses)
    ngate_long = ray_pulses[0][0].ngate_long_hi
    ngate_short = ray_pulses[0][0].ngate_short_hi
    ngate = ngate_long + ngate_short

    # Elevation assumed to be flat from the very first pulse
    scan_el = ray_pulses[0][0].elevation
    scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

    # Sampling code from the CPI header
    fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
    dr = 3.0e8 / fs / 2
    r = 1.0e-3 * np.arange(0, ngate, dtype=np.single) * dr

    # Noise estimate, try azimuth 0, around 20-25 km
    ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
    ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
    samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

    # Gather the samples. Ignore phase code since we are only interested in amplitude
    for k, pulse in enumerate(ray_pulses[ia]):
        samples[k, :] = pulse.h_long_hi[ir:er]
    noise = np.mean(np.abs(samples)) ** 2
    print('Noise estimate in 16-bit ADU: {:.4f}'.format(noise))
    
    s = np.zeros((naz, ngate), dtype=np.single)

    # Go through the pulses
    a = np.zeros(len(ray_pulses), dtype=np.single)
    for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
        p = np.zeros((len(pulses), ngate), dtype=np.csingle)
        a[k] = pulses[0].azimuth
        for j, pulse in enumerate(pulses):
            p[j, ngate_short:] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
            p[j, :ngate_short] = pulse.h_short_hi * np.exp(-1j * pulse.phase_h_short)
        s[k, :] = np.mean(np.abs(p) ** 2, axis=0)

    # Signal
    s -= noise
    s[s <= 0] = 1.0e-6                                    # Avoid log(0)
    snr = 10 * np.log10(s / noise)                        # Signal-to-noise ratio
    z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
    z[:, :ngate_short] += 30                              # 30-dB on the short waveform, perhaps?

    # Thresholding
    m = snr < 0
    z[m] = np.nan
    
    # Coordinate
    ce = np.cos(np.deg2rad(scan_el))
    rr, aa = np.meshgrid(r, np.deg2rad(a))
    xx = rr * ce * np.sin(aa)
    yy = rr * ce * np.cos(aa)

    # Plot
    plt.figure(figsize=(5.2, 4), dpi=200)
    plt.pcolormesh(xx[:, :500], yy[:, :500], z[:, :500], cmap=cmap)
    # plt.plot(x_turb, y_turb, 'xk')
    plt.clim((-32, 96))
    # plt.xlim((-13, 5))
    # plt.ylim((3, 18))
    # plt.xlim((-25, 10))
    # plt.ylim((0, 35))

    plt.xlim((-20, 20))
    plt.ylim((-20, 20))

    plt.colorbar()
    plt.grid(linewidth=0.25)
    plt.title(os.path.basename(file), fontsize=8)
    im = os.path.expanduser('~/Downloads/akita/{}.png'.format(os.path.basename(file)[:-4]))
    print(im) 
    plt.savefig(im)
    plt.close()

/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/15/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat
Data read in 9.92 s
Noise estimate in 16-bit ADU: 20.6689
/Users/boonleng/Downloads/akita/20200614_150015.256007-87-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/15/20200614_150608.563244-9B-02.iqData.XXXX.AKITA.dat
Data read in 9.40 s
Noise estimate in 16-bit ADU: 21.3115
/Users/boonleng/Downloads/akita/20200614_150608.563244-9B-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/15/20200614_151201.636704-AF-02.iqData.XXXX.AKITA.dat
Data read in 9.39 s
Noise estimate in 16-bit ADU: 20.7982
/Users/boonleng/Downloads/akita/20200614_151201.636704-AF-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/15/20200614_151754.907843-C3-02.iqData.XXXX.AKITA.dat
Data read in 9.26 s
Noise estimate in 16-bit ADU: 20.6486
/Users/boonleng/Downloads/akita/20200614_151754.907843-C3-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2

/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/18/20200614_182020.793455-2F-02.iqData.XXXX.AKITA.dat
Data read in 9.78 s
Noise estimate in 16-bit ADU: 21.9018
/Users/boonleng/Downloads/akita/20200614_182020.793455-2F-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/18/20200614_182614.071282-43-02.iqData.XXXX.AKITA.dat
Data read in 9.32 s
Noise estimate in 16-bit ADU: 22.0021
/Users/boonleng/Downloads/akita/20200614_182614.071282-43-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/18/20200614_183206.906113-57-02.iqData.XXXX.AKITA.dat
Data read in 9.14 s
Noise estimate in 16-bit ADU: 22.4511
/Users/boonleng/Downloads/akita/20200614_183206.906113-57-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/18/20200614_183759.631191-6B-02.iqData.XXXX.AKITA.dat
Data read in 9.07 s
Noise estimate in 16-bit ADU: 21.6490
/Users/boonleng/Downloads/akita/20200614_183759.631191-6B-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2

/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/21/20200614_214025.738626-D7-02.iqData.XXXX.AKITA.dat
Data read in 9.24 s
Noise estimate in 16-bit ADU: 20.8857
/Users/boonleng/Downloads/akita/20200614_214025.738626-D7-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/21/20200614_214618.814880-EB-02.iqData.XXXX.AKITA.dat
Data read in 9.07 s
Noise estimate in 16-bit ADU: 20.6612
/Users/boonleng/Downloads/akita/20200614_214618.814880-EB-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/21/20200614_215212.575009-FF-02.iqData.XXXX.AKITA.dat
Data read in 9.03 s
Noise estimate in 16-bit ADU: 20.6399
/Users/boonleng/Downloads/akita/20200614_215212.575009-FF-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.14/21/20200614_215805.355001-13-02.iqData.XXXX.AKITA.dat
Data read in 8.94 s
Noise estimate in 16-bit ADU: 20.3766
/Users/boonleng/Downloads/akita/20200614_215805.355001-13-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2

/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/01/20200615_010033.388415-7F-02.iqData.XXXX.AKITA.dat
Data read in 9.33 s
Noise estimate in 16-bit ADU: 20.9273
/Users/boonleng/Downloads/akita/20200615_010033.388415-7F-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/01/20200615_010626.606319-93-02.iqData.XXXX.AKITA.dat
Data read in 9.57 s
Noise estimate in 16-bit ADU: 20.9386
/Users/boonleng/Downloads/akita/20200615_010626.606319-93-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/01/20200615_011218.995915-A7-02.iqData.XXXX.AKITA.dat
Data read in 9.50 s
Noise estimate in 16-bit ADU: 20.6755
/Users/boonleng/Downloads/akita/20200615_011218.995915-A7-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/01/20200615_011811.675375-BB-02.iqData.XXXX.AKITA.dat
Data read in 9.29 s
Noise estimate in 16-bit ADU: 23.3306
/Users/boonleng/Downloads/akita/20200615_011811.675375-BB-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2

/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/23/20200615_232048.253486-4A-02.iqData.XXXX.AKITA.dat
Data read in 9.42 s
Noise estimate in 16-bit ADU: 21.1867
/Users/boonleng/Downloads/akita/20200615_232048.253486-4A-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/23/20200615_232640.960321-5E-02.iqData.XXXX.AKITA.dat
Data read in 9.30 s
Noise estimate in 16-bit ADU: 20.6737
/Users/boonleng/Downloads/akita/20200615_232640.960321-5E-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/23/20200615_233234.516959-72-02.iqData.XXXX.AKITA.dat
Data read in 9.71 s
Noise estimate in 16-bit ADU: 21.1969
/Users/boonleng/Downloads/akita/20200615_233234.516959-72-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.15/23/20200615_233827.730795-86-02.iqData.XXXX.AKITA.dat
Data read in 9.54 s
Noise estimate in 16-bit ADU: 20.6083
/Users/boonleng/Downloads/akita/20200615_233827.730795-86-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2

/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/02/20200616_024054.386972-F2-02.iqData.XXXX.AKITA.dat
Data read in 9.45 s
Noise estimate in 16-bit ADU: 20.3298
/Users/boonleng/Downloads/akita/20200616_024054.386972-F2-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/02/20200616_024647.323336-06-02.iqData.XXXX.AKITA.dat
Data read in 9.61 s
Noise estimate in 16-bit ADU: 20.7337
/Users/boonleng/Downloads/akita/20200616_024647.323336-06-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/02/20200616_025240.421514-1A-02.iqData.XXXX.AKITA.dat
Data read in 9.89 s
Noise estimate in 16-bit ADU: 20.9993
/Users/boonleng/Downloads/akita/20200616_025240.421514-1A-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/02/20200616_025833.939371-2E-02.iqData.XXXX.AKITA.dat
Data read in 10.22 s
Noise estimate in 16-bit ADU: 20.6026
/Users/boonleng/Downloads/akita/20200616_025833.939371-2E-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/

/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/06/20200616_060102.485901-9A-02.iqData.XXXX.AKITA.dat
Data read in 9.76 s
Noise estimate in 16-bit ADU: 21.3683
/Users/boonleng/Downloads/akita/20200616_060102.485901-9A-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/06/20200616_060655.982210-AE-02.iqData.XXXX.AKITA.dat
Data read in 9.85 s
Noise estimate in 16-bit ADU: 21.0724
/Users/boonleng/Downloads/akita/20200616_060655.982210-AE-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/06/20200616_061249.835315-C2-02.iqData.XXXX.AKITA.dat
Data read in 9.72 s
Noise estimate in 16-bit ADU: 19.8428
/Users/boonleng/Downloads/akita/20200616_061249.835315-C2-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/06/20200616_061843.119769-D6-02.iqData.XXXX.AKITA.dat
Data read in 9.78 s
Noise estimate in 16-bit ADU: 20.8864
/Users/boonleng/Downloads/akita/20200616_061843.119769-D6-02.iqData.XXXX.AKITA.png
/Volumes/Transcend/akita/IQ/IQdata/2